In [1]:
# Version Check
import sys
import tensorflow as tf
import tensorflow_datasets as tfds
import PIL
import pandas as pd
import numpy as np
import scipy
print("python", sys.version)
print("tensorflow", tf.__version__)
print("tensorflow-datasets", tfds.__version__)
print("Pillow", PIL.__version__)
print("pandas", pd.__version__)
print("numpy", np.__version__)
print("scipy", scipy.__version__)
print()
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Built with GPU support:", tf.test.is_built_with_gpu_support())

python 3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]
tensorflow 2.6.2
tensorflow-datasets 4.4.0
Pillow 8.3.2
pandas 1.3.3
numpy 1.19.5
scipy 1.7.1

Num GPUs Available: 1
Built with CUDA: True
Built with GPU support: True


In [3]:
import matplotlib.pylab as plt
import cv2
from tqdm.notebook import tqdm

In [4]:
# parameters
img_shape = (28, 28, 1)
z_dim = 100
row_num = 8
col_num = 8
batch_size = row_num * col_num
epoch_num = 10
learning_rate = 0.0001
class_num = 10

In [7]:
# data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

In [13]:
# discriminator model

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization, Dense, Reshape

i = Input(shape=img_shape)
out = Conv2D(16, 3, 2, padding='same')(i)  # num_filters, filter size(3, 3), strides(2, 2)
out = Conv2D(32, 3, 2, padding='same')(out)
out = Conv2D(64, 3, 2, padding='same')(out)
out = Flatten()(out)
out = BatchNormalization()(out)
out = Dense(1024, activation='tanh')(out)
out = Dense(1, activation='sigmoid')(out)

d_model = Model(inputs=[i], outputs=[out])

d_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049

In [14]:
# generator model

from tensorflow.keras.layers import Reshape, Conv2DTranspose, Activation

i = Input(shape=(z_dim, ))
out = Dense(1024, activation='tanh')(i)
out = Dense(7 * 7 * 32, activation='tanh')(out)
out = BatchNormalization()(out)
out = Reshape((7, 7, 32))(out)
out = Conv2DTranspose(16, 3, 2, padding='same')(out)
out = Conv2DTranspose(1, 3, 2, padding='same')(out)
out = Activation('sigmoid')(out)

g_model = Model(inputs=[i], outputs=[out])

g_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              103424    
_________________________________________________________________
dense_7 (Dense)              (None, 1568)              1607200   
_________________________________________________________________
batch_normalization_3 (Batch (None, 1568)              6272      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 16)        4624      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 1)         145 

In [15]:
# optimizer
opt = tf.keras.optimizers.Adam(learning_rate)

# discriminator loss function
d_loss = tf.keras.losses.BinaryCrossentropy()

In [17]:
fcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('hjk_gan_mnist.avi', fcc, 1.0, (28 * row_num, 28 * col_num))

batch_count =x_train.shape[0] // batch_size

for e in range(epoch_num):
    for _ in tqdm(range(batch_count)):
        
        z = np.random.uniform(-1.0, 1.0, (batch_size, z_dim))
        f_img = g_model.predict(z)
        f_label = np.zeros((batch_size, 1))
        
        with tf.GradientTape() as tape:
            pred = d_model(f_img)
            loss = d_loss(f_label, pred)
        vars = d_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))
        
        batch_num=np.random.randint(0,  x_train.shape[0],  size=batch_size)
        r_img = x_train[batch_num]
        r_label = np.ones((batch_size, 1))
        
        with tf.GradientTape() as tape:
            pred = d_model(r_img)
            loss = d_loss(r_label, pred)
        vars = d_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))

        with tf.GradientTape() as tape:
            f_img = g_model(z)
            pred = d_model(f_img)
            loss = d_loss(r_label, pred)
        vars = g_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))     
        
        sample_img = np.zeros((28*row_num, 28 * col_num))
        f_img = np.resize(f_img, (row_num, col_num, 28, 28))
        for i in range(row_num):
            for j in range(col_num):
                sample_img[i * 28:i * 28 +28, j * 28:j * 28 +28] = f_img[i, j, :, :]
        sample_img = np.uint8(sample_img * 255.)
        sample_img = cv2.applyColorMap(sample_img, cv2.COLORMAP_HOT)
        out.write(sample_img)
    
    print(e, "완료")
    
out.release()

  0%|          | 0/937 [00:00<?, ?it/s]

0 완료


  0%|          | 0/937 [00:00<?, ?it/s]

1 완료


  0%|          | 0/937 [00:00<?, ?it/s]

...

7 완료


  0%|          | 0/937 [00:00<?, ?it/s]

8 완료


  0%|          | 0/937 [00:00<?, ?it/s]

9 완료


In [18]:
i = tf.keras.Input(shape=img_shape)
l_i = tf.keras.Input(shape=(1, ), dtype=tf.int32)
l_out = tf.one_hot(l_i, class_num)
l_out = tf.keras.layers.Dense(28*28*1)(l_out)
l_out = tf.keras.layers.Reshape((28, 28, 1))(l_out) 
out = tf.keras.layers.Add()([i, l_out])
out = tf.keras.layers.Conv2D(16, 3, 2, padding='same')(out)
out = tf.keras.layers.Conv2D(32, 3, 2, padding='same')(out)
out = tf.keras.layers.Conv2D(64, 3, 2, padding='same')(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(1024, activation='tanh')(out)
out = tf.keras.layers.Dense(1, activation='sigmoid')(out)
d_model = tf.keras.Model(inputs=[i, l_i],  outputs=[out])

i=tf.keras.Input(shape=(z_dim, ))
l_i = tf.keras.Input(shape=(1, ), dtype=tf.int32)
l_out = tf.one_hot(l_i, class_num)
l_out = tf.keras.layers.Dense(z_dim)(l_out)
l_out = tf.keras.layers.Reshape((z_dim, ))(l_out) 
out = tf.keras.layers.Add()([i, l_out])
out = tf.keras.layers.Dense(1024, activation='tanh')(out)
out = tf.keras.layers.Dense(7*7*32, activation='tanh')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Reshape((7, 7, 32))(out)
out = tf.keras.layers.Conv2DTranspose(16, 3, 2, padding='same')(out)
out = tf.keras.layers.Conv2DTranspose(1, 3, 2, padding='same')(out)
out = tf.keras.layers.Activation('sigmoid')(out)
g_model = tf.keras.Model(inputs=[i, l_i],  outputs=[out])

fcc=cv2.VideoWriter_fourcc(*'DIVX')
out=cv2.VideoWriter('cgan_mnist.avi', fcc, 10.0, (28*row_num, 28*col_num))

batch_count =x_train.shape[0]//batch_size

for e in range(epoch_num):
    for _ in tqdm(range(batch_count)):
        
        z = np.random.uniform(-1.0, 1.0, (batch_size, z_dim))
        f_y = np.random.randint(0, class_num, size=batch_size)
        f_y = np.reshape(f_y, (batch_size, 1))
        f_img = g_model.predict([z, f_y])
        f_label = np.zeros((batch_size, 1))
        
        with tf.GradientTape() as tape:
            pred = d_model([f_img, f_y])
            loss = d_loss(f_label, pred)
        vars = d_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))
        
        batch_num=np.random.randint(0, x_train.shape[0], size=batch_size)
        r_img = x_train[batch_num]
        r_y = y_train[batch_num]
        r_label = np.ones((batch_size, 1))

        with tf.GradientTape() as tape:
            pred = d_model([r_img, r_y])
            loss = d_loss(r_label, pred)
        vars = d_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))

        f_y=[i%class_num for i in range(batch_size)]
        f_y = np.reshape(f_y, (batch_size, 1))

        with tf.GradientTape() as tape:
            f_img = g_model([z, f_y])
            pred = d_model([f_img, f_y])
            loss = d_loss(r_label, pred)
        vars = g_model.trainable_variables
        grad = tape.gradient(loss,  vars)
        opt.apply_gradients(zip(grad,  vars))     
        
        sample_img = np.zeros((28*row_num, 28*col_num))
        f_img = np.resize(f_img, (row_num, col_num, 28, 28))
        for i in range(row_num):
            for j in range(col_num):
                sample_img[i * 28:i * 28 +28, j * 28:j * 28 +28] = f_img[i, j, :, :]
        sample_img = np.uint8(sample_img * 255.)
        sample_img = cv2.applyColorMap(sample_img, cv2.COLORMAP_HOT)
        out.write(sample_img)
    
    print(e, "완료")
out.release()

  0%|          | 0/937 [00:00<?, ?it/s]

0 완료


  0%|          | 0/937 [00:00<?, ?it/s]

1 완료


  0%|          | 0/937 [00:00<?, ?it/s]

...

7 완료


  0%|          | 0/937 [00:00<?, ?it/s]

8 완료


  0%|          | 0/937 [00:00<?, ?it/s]

9 완료
